In [1]:
import pyodbc
import pandas as pd
from sqlalchemy import create_engine,text
import urllib
import numpy as np
import math

In [2]:

# this code for make a connection on data base by ms sql server
# Connection string (adjust SERVER, DATABASE, and authentication as needed)
params = urllib.parse.quote_plus(
    r"DRIVER={ODBC Driver 17 for SQL Server};"
    r"SERVER=localhost\SQLEXPRESS;"   # replace SQLEXPRESS with your actual instance name
    r"DATABASE=TrainTicketsDB;"               # replace master by your data base name now it is TrainTicketsDB
    r"Trusted_Connection=yes;"
)

engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")

with engine.connect() as conn:
    result = conn.execute(text("SELECT @@SERVERNAME;"))
    print(result.fetchone())

('DESKTOP-D9VQ18F\\SQLEXPRESS',)


In [6]:


# Switch to your database (create it if needed)

with engine.begin() as conn:
    conn.execute(text("""
        IF DB_ID('TrainTicketsDB') IS NULL CREATE DATABASE TrainTicketsDB;
                USE TrainTicketsDB;
        """
    ))

# 1. Passenger table
with engine.begin() as conn:
    conn.execute(text("""
            IF OBJECT_ID('Passenger', 'U') IS NOT NULL DROP TABLE Passenger;
            CREATE TABLE Passenger (
            Passenger_ID INT IDENTITY(1,1) PRIMARY KEY,
            Railcard_Type VARCHAR(20) CHECK (Railcard_Type IN ('None', 'Adult', 'Senior', 'Disabled'))
        );
        """
    ))
# 2. Purchase table
with engine.begin() as conn:
    conn.execute(text("""
       IF OBJECT_ID('Purchase', 'U') IS NOT NULL DROP TABLE Purchase;
    CREATE TABLE Purchase (
    Transaction_ID INT IDENTITY(1,1) PRIMARY KEY,
    Passenger_ID INT NOT NULL,
    Date_of_Purchase DATE NOT NULL,
    Time_of_Purchase TIME NOT NULL,
    Purchase_Type VARCHAR(20) CHECK (Purchase_Type IN ('Online', 'Station')),
    Payment_Method VARCHAR(20) CHECK (Payment_Method IN ('Contactless', 'Credit Card', 'Debit Card')),
    Price DECIMAL(10,2) NOT NULL,
    FOREIGN KEY (Passenger_ID) REFERENCES Passenger(Passenger_ID)
);
        """
    ))
# 3. Ticket table
with engine.begin() as conn:
    conn.execute(text("""
       IF OBJECT_ID('Ticket', 'U') IS NOT NULL DROP TABLE Ticket;
    CREATE TABLE Ticket (
    Ticket_ID INT IDENTITY(1,1) PRIMARY KEY,
    Transaction_ID INT NOT NULL,
    Ticket_Class VARCHAR(20) CHECK (Ticket_Class IN ('Standard', 'First')),
    Ticket_Type VARCHAR(20) CHECK (Ticket_Type IN ('Advance', 'Off-Peak', 'Anytime')),
    FOREIGN KEY (Transaction_ID) REFERENCES Purchase(Transaction_ID)
);
        """
    ))
# 4. Journey table
with engine.begin() as conn:
    conn.execute(text("""
      IF OBJECT_ID('Journey', 'U') IS NOT NULL DROP TABLE Journey;
    CREATE TABLE Journey (
    Journey_ID INT IDENTITY(1,1) PRIMARY KEY,
    Ticket_ID INT NOT NULL,
    Date_of_Journey DATE NOT NULL,
    Departure_Station VARCHAR(100) NOT NULL,
    Arrival_Destination VARCHAR(100) NOT NULL,
    Departure_Time TIME NOT NULL,
    Arrival_Time TIME NOT NULL,
    Actual_Arrival_Time TIME,
    Journey_Status VARCHAR(20) CHECK (Journey_Status IN ('On-time', 'Delayed', 'Cancelled')),
    Reason_for_Delay VARCHAR(255),
    FOREIGN KEY (Ticket_ID) REFERENCES Ticket(Ticket_ID)
);
        """
    ))
# 5. Refund table
with engine.begin() as conn:
    conn.execute(text("""
       IF OBJECT_ID('Refund', 'U') IS NOT NULL DROP TABLE Refund;
    CREATE TABLE Refund (
    Refund_ID INT IDENTITY(1,1) PRIMARY KEY,
    Journey_ID INT NOT NULL,
    Refund_Request BIT NOT NULL,
    FOREIGN KEY (Journey_ID) REFERENCES Journey(Journey_ID)
);
        """
    ))


conn.commit()
conn.close()

print("SQL Server schema created successfully in TrainTicketsDB!")


SQL Server schema created successfully in TrainTicketsDB!


### before run the code below reconnect to the server from the cell above 

In [3]:
df = pd.read_csv("../datasets/railway.csv" , sep =",")
valid_railcards = {"None", "Adult", "Senior", "Disabled"}
valid_purchase_type = {"Online", "Station"}
valid_payment = {"Contactless", "Credit Card", "Debit Card"}
valid_ticket_class = {"Standard", "First"}
valid_ticket_type = {"Advance", "Off-Peak", "Anytime"}
valid_journey_status = {"On-time", "Delayed", "Cancelled"} 
print(df.head())

            Transaction ID Date of Purchase Time of Purchase Purchase Type  \
0  da8a6ba8-b3dc-4677-b176       2023-12-08         12:41:11        Online   
1  b0cdd1b0-f214-4197-be53       2023-12-16         11:23:01       Station   
2  f3ba7a96-f713-40d9-9629       2023-12-19         19:51:27        Online   
3  b2471f11-4fe7-4c87-8ab4       2023-12-20         23:00:36       Station   
4  2be00b45-0762-485e-a7a3       2023-12-27         18:22:56        Online   

  Payment Method Railcard Ticket Class Ticket Type  Price  \
0    Contactless    Adult     Standard     Advance     43   
1    Credit Card    Adult     Standard     Advance     23   
2    Credit Card      NaN     Standard     Advance      3   
3    Credit Card      NaN     Standard     Advance     13   
4    Contactless      NaN     Standard     Advance     76   

       Departure Station    Arrival Destination Date of Journey  \
0      London Paddington  Liverpool Lime Street      2024-01-01   
1     London Kings Cross      

In [4]:
# Clean up column names
df.columns = df.columns.str.strip()       # remove spaces
df.columns = df.columns.str.replace('\ufeff', '')  # remove BOM if present
print(df.columns.tolist())                # check actual column names
df = df.where(pd.notnull(df), None)


['Transaction ID', 'Date of Purchase', 'Time of Purchase', 'Purchase Type', 'Payment Method', 'Railcard', 'Ticket Class', 'Ticket Type', 'Price', 'Departure Station', 'Arrival Destination', 'Date of Journey', 'Departure Time', 'Arrival Time', 'Actual Arrival Time', 'Journey Status', 'Reason for Delay', 'Refund Request']


In [5]:
def clean_value(value):
    """Convert NaN/None to SQL NULL safely"""
    if value is None or (isinstance(value, float) and math.isnan(value)):
        return None
    return value

with engine.begin() as conn:
    for _, row in df.iterrows():
        # ------------------
        # 1. Insert Passenger
        # ------------------
        railcard = clean_value(row["Railcard"])
        if railcard not in valid_railcards:
            railcard = "None"  # default if invalid

        passenger_id = conn.execute(
            text("""
                INSERT INTO Passenger (Railcard_Type)
                OUTPUT INSERTED.Passenger_ID
                VALUES (:railcard)
            """),
            {"railcard": railcard}
        ).scalar()

        # ------------------
        # 2. Insert Purchase
        # ------------------
        purchase_type = clean_value(row["Purchase Type"])
        if purchase_type not in valid_purchase_type:
            purchase_type = "Online"

        payment_method = clean_value(row["Payment Method"])
        if payment_method not in valid_payment:
            payment_method = "Credit Card"

        price = clean_value(row["Price"])
        if price is None:
            price = 0.00  # safe fallback

        transaction_id = conn.execute(
            text("""
                INSERT INTO Purchase (Passenger_ID, Date_of_Purchase, Time_of_Purchase,
                                      Purchase_Type, Payment_Method, Price)
                OUTPUT INSERTED.Transaction_ID
                VALUES (:passenger_id, :date_purchase, :time_purchase,
                        :purchase_type, :payment_method, :price)
            """),
            {
                "passenger_id": passenger_id,
                "date_purchase": row["Date of Purchase"],
                "time_purchase": row["Time of Purchase"],
                "purchase_type": purchase_type,
                "payment_method": payment_method,
                "price": price,
            }
        ).scalar()

        # ------------------
        # 3. Insert Ticket
        # ------------------
        ticket_class = clean_value(row["Ticket Class"])
        if ticket_class not in valid_ticket_class:
            ticket_class = "Standard"

        ticket_type = clean_value(row["Ticket Type"])
        if ticket_type not in valid_ticket_type:
            ticket_type = "Anytime"

        ticket_id = conn.execute(
            text("""
                INSERT INTO Ticket (Transaction_ID, Ticket_Class, Ticket_Type)
                OUTPUT INSERTED.Ticket_ID
                VALUES (:transaction_id, :ticket_class, :ticket_type)
            """),
            {
                "transaction_id": transaction_id,
                "ticket_class": ticket_class,
                "ticket_type": ticket_type,
            }
        ).scalar()

        # ------------------
        # 4. Insert Journey
        # ------------------
        journey_status = clean_value(row["Journey Status"])
        if journey_status not in valid_journey_status:
            journey_status = "On-time"  # default safe value

        reason_for_delay = clean_value(row["Reason for Delay"])
        if journey_status not in {"Delayed", "Cancelled"}:
            reason_for_delay = None  # only keep if delay/cancelled

        journey_id = conn.execute(
            text("""
                INSERT INTO Journey (Ticket_ID, Date_of_Journey, Departure_Station, Arrival_Destination,
                                     Departure_Time, Arrival_Time, Actual_Arrival_Time,
                                     Journey_Status, Reason_for_Delay)
                OUTPUT INSERTED.Journey_ID
                VALUES (:ticket_id, :date_journey, :departure_station, :arrival_destination,
                        :departure_time, :arrival_time, :actual_arrival_time,
                        :journey_status, :reason_for_delay)
            """),
            {
                "ticket_id": ticket_id,
                "date_journey": row["Date of Journey"],
                "departure_station": row["Departure Station"],
                "arrival_destination": row["Arrival Destination"],
                "departure_time": row["Departure Time"],
                "arrival_time": row["Arrival Time"],
                "actual_arrival_time": clean_value(row["Actual Arrival Time"]),
                "journey_status": journey_status,
                "reason_for_delay": reason_for_delay,
            }
        ).scalar()

        # ------------------
        # 5. Insert Refund
        # ------------------
        refund_request = clean_value(row.get("Refund Request"))
        if refund_request in ["Yes", "yes", 1, True]:
            refund_request = 1
        else:
            refund_request = 0

        conn.execute(
            text("""
                INSERT INTO Refund (Journey_ID, Refund_Request)
                VALUES (:journey_id, :refund_request)
            """),
            {"journey_id": journey_id, "refund_request": refund_request}
        )
